## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

## Imports

In [1]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.model_selection import train_test_split
from torch import nn

## Variáveis Globais e Inicialização

In [2]:
# Global variables

# Vocabulary
vocab_size = 5000
context_size = 5
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 128
epochs = 10
lr = 0.1

# Model
embedding_dim = 256
hidden_dim = 128

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

## Faz download e carrega o dataset

In [4]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [5]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [6]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [7]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

Quem conhece a vegetação de nossa terra desde a parasita mimosa até o cedro gigante; quem no reino animal desce do tigre e do tapir, symbolos da ferocidade e da força, até o lindo beija-flôr e o insecto dourado; quem olha este céo que passa do mais puro anil aos reflexos bronzeados que annuncião as grandes borrascas; quem viu sob a verde pellucia da relva esmaltada de flôres que cobre as nossas varzeas deslisar mil reptis que levão a morte n'um atomo de veneno, comprehende o que Alvaro sentio.

--Tu viverás!

O indio tinha a sua idéa.

Por fim ergueu a cabeça. Seus companheiros esperavão uma palavra sua como o oraculo do seu destino; preparárão-se para ouvi-lo.

--O Sr. Alvaro e os outros.

Number of paragraphs: 4892


4892

## Análise do dataset

In [8]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [9]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [10]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 5000


In [11]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 4383, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [12]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [13]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 59646
Validation dataset samples: 16215


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[  47,    6,   73,    2,   67],
        [  12,    5, 3157,    9,  166],
        [ 365,  921,   16,  638, 3704],
        [  14,  665, 2104,    2,    7],
        [  11,  141,    9,   40,   12],
        [   4,   32,    2,    7,   16],
        [4815,  152,    6,    4,   15],
        [ 124,   23,   48,   10,  320],
        [ 832,   36, 1398,    1, 4413],
        [   9,  687, 1394,    8, 4402],
        [  16,    3,  658,   24,  207],
        [  62,   13,  372,  627, 4597],
        [2395,   11,  337,    4,   52],
        [4337,    2, 1627,    7,  135],
        [1564,   21,    4,  203,   27],
        [  11,    4,   65, 1576,    9],
        [  98, 4515,    2,   11,    4],
        [  11,  542,    2,  457,    8],
        [  78,  605,  769,  142,   63],
        [   1,    1,    1,    1,    1],
        [   2,  948,  161,    2,  335],
        [  47,    6,   73,    2,   17],
        [  72, 2062,    9,   15,   85],
        [   6, 2154,    2,  110,    5],
        [  52, 3936,    2,  180,    1],

## Model

##### Implementação das camadas de self-attention

In [15]:
# Loop implementation
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]

embeddings = nn.Embedding(vocab_size+1, embedding_dim)
embeds = embeddings(input)

batch_size, sequence_length, embedding_dimension = embeds.size()
print(f'Initial Embedding dimensions: {batch_size}, {sequence_length}, {embedding_dimension}')

# New embeddings
E = []
for q in range(sequence_length):
    scores = []
    C_q = embeds[:,q,:]
    for k in range(sequence_length):        
        C_k = embeds[:,k,:]
        
        # Scalar product between the embeddings
        score = torch.matmul(C_q, C_k.transpose(0,1))            
        scores.append(score)
    
    # Stack the matrixes resulting from the scalar products
    scores = torch.stack(scores, dim=1)
    probs = F.softmax(scores, dim=1)

    attn_embedding = torch.zeros_like(C_q)

    #for v, p in zip(range(sequence_length), probs):
    #    C_v = embeds[:,v,:]
    #    print(C_v.size())
    #    print(p.size())

    #for v, p in zip(embeds, probs):
    #    #attn_embedding += v * p2

Initial Embedding dimensions: 128, 5, 256


In [16]:
class SelfAttention_Matrix(nn.Module):
  def __init__(self, embed_dim):
    super().__init__()
    self.embed_dim = embed_dim

  def forward(self, inputs):
    scores = torch.matmul(inputs, inputs.transpose(1, 2))
    probs = F.softmax(scores, dim=-1)
    new_embedding = torch.matmul(probs, inputs)
    return new_embedding

In [17]:
class BengioModel_SelfAttentionMatrix(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel_SelfAttentionMatrix, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.attention = SelfAttention_Matrix(embedding_dim)        
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        x = torch.stack(torch.unbind(embeds, dim=1), dim=1)
        # Camada de autoatenção
        attention  = self.attention(x)
        # Flatten embeddings
        embeds = embeds.view(attention.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

In [18]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

## Funções de Treinamento e Avaliação do Modelo

In [19]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has a total of {total_params:,} parameters.')

In [20]:
def initial_eval(model):
    # Initial Perplexity and Loss
    # Before training
    model.eval()

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item()

    loss /= len(train_loader)
    perp = torch.exp(torch.tensor(loss))

    print(f'Initial Loss: {loss:.4f}')
    print(f'Initial Perplexity: {perp:.4f}')

In [21]:
def train(model, criterion, optimizer):
      # Training Loop
      model.train()
      for epoch in range(epochs):

            epoch_start = time.time()
            # Metrics
            epoch_loss = 0
            epoch_correct = 0
            epoch_samples = 0

            for inputs, targets in train_loader:
                  inputs = inputs.to(device)  # Move input data to the device
                  targets = targets.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  loss = criterion(outputs, targets)

                  # Backward pass and optimization
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

                  # Loss
                  epoch_loss += loss.item()

                  # Predicted
                  _, predicted = torch.max(outputs, 1)
                  epoch_correct += (predicted == targets).sum().item()
                  epoch_samples += targets.size(0)

            # Calculate average loss and accuracy for epoch
            avg_loss = epoch_loss / len(train_loader)
            acc = epoch_correct / epoch_samples

            # Perplexity
            perp = torch.exp(torch.tensor(avg_loss))

            epoch_end = time.time()
            epoch_time = epoch_end - epoch_start
            # Print epoch statistics
            print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


In [22]:
def eval(model, criterion):
    model.eval()

    loss_sum = 0
    total_sum = 0
    correct_sum = 0
    eval_round = 0

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)      
            loss_sum += loss

            # Get the predicted labels
            _, predicted = torch.max(outputs, 1)

            total_sum += targets.size(0)
            correct_sum += (predicted == targets).sum().item()
            eval_round += 1

    # Calculate accuracy
    acc = 100 * correct_sum / total_sum

    # Calculate average perplexity
    average_loss = loss_sum / len(val_loader)
    average_perplexity = torch.exp(average_loss)

    print(f'Test Accuracy: {acc:.2f}%')
    print(f'Average Loss: {average_loss:.2f}')
    print(f'Average Perplexity: {average_perplexity:.2f}')

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Sem camada de atenção

In [24]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)
print("Model without Self Attention:")
print()
count_parameters(model)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr)

model.to(device)

print()
print("Training Start")
print()
train(model, criterion, optimizer)

print()
print("Evaluation Start")
print()
eval(model, criterion)

Model without Self Attention:

The model has a total of 2,089,353 parameters.

Training Start

Epoch [1/10], Time:4.36, Loss: 6.4602, Accuracy: 0.11%, Perplexity: 639.2007
Epoch [2/10], Time:4.39, Loss: 5.5800, Accuracy: 0.15%, Perplexity: 265.0808
Epoch [3/10], Time:4.27, Loss: 5.1956, Accuracy: 0.18%, Perplexity: 180.4675
Epoch [4/10], Time:4.59, Loss: 4.9048, Accuracy: 0.20%, Perplexity: 134.9389
Epoch [5/10], Time:4.40, Loss: 4.6614, Accuracy: 0.22%, Perplexity: 105.7854
Epoch [6/10], Time:4.28, Loss: 4.4434, Accuracy: 0.24%, Perplexity: 85.0617
Epoch [7/10], Time:4.57, Loss: 4.2396, Accuracy: 0.26%, Perplexity: 69.3808
Epoch [8/10], Time:5.21, Loss: 4.0484, Accuracy: 0.28%, Perplexity: 57.3036
Epoch [9/10], Time:4.73, Loss: 3.8654, Accuracy: 0.30%, Perplexity: 47.7223
Epoch [10/10], Time:4.37, Loss: 3.6916, Accuracy: 0.31%, Perplexity: 40.1082

Evaluation Start

Test Accuracy: 19.25%
Average Loss: 5.30
Average Perplexity: 200.34


## 2. Com camada de atenção

In [25]:
model_attn = BengioModel_SelfAttentionMatrix(vocab_size, embedding_dim, context_size, hidden_dim)
print("Model with Self Attention:")
print()
count_parameters(model_attn)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model_attn.parameters(), lr)

model_attn.to(device)

print()
print("Training Start")
print()
train(model_attn, criterion, optimizer)

print()
print("Evaluation Start")
print()
eval(model_attn, criterion)

Model with Self Attention:

The model has a total of 2,089,353 parameters.

Training Start

Epoch [1/10], Time:4.86, Loss: 6.4468, Accuracy: 0.11%, Perplexity: 630.6918
Epoch [2/10], Time:5.02, Loss: 5.5854, Accuracy: 0.15%, Perplexity: 266.4969
Epoch [3/10], Time:4.78, Loss: 5.2055, Accuracy: 0.18%, Perplexity: 182.2745
Epoch [4/10], Time:4.87, Loss: 4.9130, Accuracy: 0.20%, Perplexity: 136.0409
Epoch [5/10], Time:4.95, Loss: 4.6676, Accuracy: 0.22%, Perplexity: 106.4436
Epoch [6/10], Time:4.80, Loss: 4.4491, Accuracy: 0.24%, Perplexity: 85.5504
Epoch [7/10], Time:4.72, Loss: 4.2463, Accuracy: 0.26%, Perplexity: 69.8433
Epoch [8/10], Time:4.82, Loss: 4.0555, Accuracy: 0.28%, Perplexity: 57.7146
Epoch [9/10], Time:4.79, Loss: 3.8739, Accuracy: 0.30%, Perplexity: 48.1315
Epoch [10/10], Time:4.65, Loss: 3.6971, Accuracy: 0.32%, Perplexity: 40.3294

Evaluation Start

Test Accuracy: 19.66%
Average Loss: 5.30
Average Perplexity: 200.76
